In [ ]:
from grid.eval import *

In [ ]:
boardsize = 5
n_workers = 8

snaps = snapshots(boardsize)
snaps = pd.concat([snaps, parameters(snaps)], 1)
snaps['nickname'] = snaps.run.str.extract('.* (.*)', expand=False) + '.' + snaps.idx.astype(str)
snaps['params'] = params(snaps)
snaps = snaps.set_index('nickname')

games, wins = load(boardsize, snaps.index)

soln = None
futures = {}
with ProcessPoolExecutor(n_workers) as pool:
    while True:
        try:
            soln = activelo.solve(games, wins, soln=soln)
        except InManifoldError:
            soln = None
            log.warning('Got a manifold error; throwing soln out')

        for key, future in list(futures.items()):
            if future.done():
                results = future.result()
                games, wins = update(games, wins, results)
                del futures[key]
                save(boardsize, games, wins)

        while len(futures) < n_workers:
            if soln is None:
                sugg = tuple(np.random.choice(games.index, (2,)))
            else:
                sugg = suggest(soln)
            
            futures[sugg] = pool.submit(evaluate, *sugg)
    
        if soln is not None:
            report(soln, games, futures)
            _, σ = arena.analysis.difference(soln, soln.μ.idxmin())
            if σ.pow(2).mean()**.5 < .01:
                break
        
snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmin())

In [ ]:
ProcessPoolExecutor??

In [ ]:
from concurrent.futures.process import _process_worker

In [ ]:
_process_worker??